In [1]:
!git clone https://github.com/MarkBridge11/RINE-extended

Cloning into 'RINE-extended'...
remote: Enumerating objects: 471, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 471 (delta 2), reused 2 (delta 0), pack-reused 461 (from 2)
Receiving objects: 100% (471/471), 100.54 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (297/297), done.
Updating files: 100% (442/442), done.


In [2]:
import sys
sys.path.append('/content/RINE-extended')

In [3]:
!pip install scikit-image imageio opencv-python kornia git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zbybklt8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zbybklt8
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=68ab70446602b7b02a043db516be527b7dfdc4ab724a2d5196adf6a76f73e75f
  Stored in directory: /tmp/pip-ephem-wheel-cache-7oz801ng/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [12]:
import sys
import os
from src.utils import get_transforms, get_our_trained_model
from PIL import Image
import torch
import numpy as np
import cv2
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_, transforms, _ = get_transforms()
model = get_our_trained_model(ncls="ldm", device=device)
model.to(device).eval();

100%|███████████████████████████████████████| 890M/890M [00:12<00:00, 72.7MiB/s]


In [6]:
real_path = "/content/RINE-extended/demo/aadlygmazf.jpg"
real_image = Image.open(real_path).convert("RGB")
real_tensor = transforms(real_image).unsqueeze(0).to(device)
real_logit = model(real_tensor)[0]
real_probability = torch.sigmoid(real_logit)
print(
    f"real image - prob. to be fake: {real_probability.detach().cpu().numpy()[0][0]*100:1.1f}%"
)

real image - prob. to be fake: 35.6%


In [7]:
# g has shape [tokens, blocks, dim]
g = torch.stack([h.output for h in model.hooks], dim=2)[0]  # [tokens, blocks, dim]

# softmax weights [1, blocks, dim]
weights = torch.softmax(model.alpha, dim=1)[0]

# elementwise product before summation
contrib = g * weights  # [tokens, blocks, dim]

# sum across dim (feature dimension) → patch/block contribution
contrib_map = contrib.sum(-1)  # [tokens, blocks]

# average over blocks if you want a single score per token
token_scores = contrib_map.mean(-1)  # [tokens]

In [8]:
grid_size = int(token_scores[:-1].shape[0] ** 0.5)  # exclude CLS
heatmap = token_scores[:-1].reshape(grid_size, grid_size).cpu().detach().numpy()

In [20]:
def get_token_importance(model, x):
    with torch.no_grad():
        _ = model.clip.encode_image(x)  # run CLIP forward to fill hooks
        g = torch.stack([h.output for h in model.hooks], dim=2)[0]  # [tokens, blocks, dim]
        g = model.proj1(g.float())  # same as in forward

    # apply alpha weights
    weights = F.softmax(model.alpha, dim=1)[0]  # [blocks, dim]
    contrib = g * weights  # [tokens, blocks, dim]

    # sum across feature dimension → relevance per token/block
    contrib_map = contrib.sum(-1)  # [tokens, blocks]

    # average over blocks → single importance score per token
    token_scores = contrib_map.mean(-1)  # [tokens]

    return token_scores.cpu().detach().numpy()

In [14]:
def tokens_to_heatmap(token_scores, image_size=(224,224)):
    n_tokens = token_scores.shape[0]
    grid_size = int(np.sqrt(n_tokens - 1))  # exclude CLS token
    patch_scores = token_scores[1:].reshape(grid_size, grid_size)  # drop CLS

    # normalize 0–1
    patch_scores = (patch_scores - patch_scores.min()) / (patch_scores.max() - patch_scores.min() + 1e-6)

    # upscale to image size
    heatmap = cv2.resize(patch_scores, image_size, interpolation=cv2.INTER_CUBIC)
    return heatmap

In [15]:
def overlay_heatmap(img, heatmap, alpha=0.5):
    cmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    cmap = cv2.cvtColor(cmap, cv2.COLOR_BGR2RGB)

    img_np = np.array(img)
    overlay = cv2.addWeighted(img_np, 1-alpha, cmap, alpha, 0)

    plt.figure(figsize=(6,6))
    plt.imshow(overlay)
    plt.axis("off")
    plt.show()

In [21]:
# get token importance
token_scores = get_token_importance(model, real_tensor)

# make heatmap
heatmap = tokens_to_heatmap(token_scores, image_size=real_image.size)

# overlay
overlay_heatmap(real_image, heatmap, alpha=0.5)

ValueError: zero-size array to reduction operation minimum which has no identity

In [10]:
# fake_path = "/content/RINE-extended/demo/zzrybysskm.jpg"
# fake_image = Image.open(fake_path).convert("RGB")
# fake_tensor = transforms(fake_image).unsqueeze(0).to(device)
# fake_logit = model(fake_tensor)[0]
# fake_probability = torch.sigmoid(fake_logit)
# print(
#     f"fake image - prob. to be fake: {fake_probability.detach().cpu().numpy()[0][0]*100:1.1f}%"
# )
